In [1]:

%%capture --no-stderr
%pip install --quiet -U langgraph langchain_community langchain_core langchain-groq

In [2]:
import os,getpass
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image
from dotenv import load_dotenv

In [3]:

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")


GROQ_API_KEY: ··········


In [4]:

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.1,
)

In [11]:
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    city: str
    interests: List[str]
    Ayaz: str

In [17]:
Ayaz_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are , Ayaz ,a helpful travel assistant. Create a day trip for {city} based on the user's interests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip."),
])

In [18]:
def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your day trip:")
    user_message = input("Your input: ")
    return {
        **state,
        "city": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)],
    }

def input_interests(state: PlannerState) -> PlannerState:
    print(f"Please enter your interests for the trip to {state['city']} (comma-separated):")
    user_message = input("Your input: ")
    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(',')],
        "messages": state['messages'] + [HumanMessage(content=user_message)],
    }

def create_Ayaz(state: PlannerState) -> PlannerState:
    print(f"Ayaz is scheduling a trip for {state['city']} based on interests: {', '.join(state['interests'])}...")
    response = llm.invoke(Ayaz_prompt.format_messages(city=state['city'], interests=", ".join(state['interests'])))
    print("\nFinal Schedule:")
    print(response.content)
    return {
        **state,
        "messages": state['messages'] + [AIMessage(content=response.content)],
        "Ayaz": response.content,
    }

In [19]:
workflow = StateGraph(PlannerState)

workflow.add_node("input_city", input_city)
workflow.add_node("input_interests", input_interests)
workflow.add_node("create_Ayaz", create_Ayaz)

workflow.set_entry_point("input_city")

workflow.add_edge("input_city", "input_interests")
workflow.add_edge("input_interests", "create_Ayaz")
workflow.add_edge("create_Ayaz", END)

app = workflow.compile()

In [20]:
def run_travel_planner(user_request: str):
    print(f"Initial Request: {user_request}\n")
    state = {
        "messages": [HumanMessage(content=user_request)],
        "city": "",
        "interests": [],
        "Ayaz": "",
    }

    for output in app.stream(state):
        pass  # The nodes themselves now handle all printing

In [21]:
user_request = "I want to plan a day trip."
run_travel_planner(user_request)

Initial Request: I want to plan a day trip.

Please enter the city you want to visit for your day trip:
Your input: karachi
Please enter your interests for the trip to karachi (comma-separated):
Your input: history, food, sea
Ayaz is scheduling a trip for karachi based on interests: history, food, sea...

Final Schedule:
I'd be happy to help you plan a day trip in Karachi. Based on your interests in history, food, and the sea, I've created a brief itinerary for you:

**Day Trip Itinerary: "Karachi's Coastal Heritage"**

* **9:00 AM - 10:00 AM:** Start the day with a visit to the **Quaid-e-Azam Museum** (Flagstaff House), a historic house museum showcasing the life and legacy of Muhammad Ali Jinnah, the founder of Pakistan.
* **10:30 AM - 12:00 PM:** Head to the **Mohatta Palace**, a stunning example of Rajasthani architecture that now serves as a museum. Explore the palace's beautiful gardens, fountains, and exhibits that highlight Karachi's rich cultural heritage.
* **1:00 PM - 2:30 P